# World Cup Prediction Model
### By: David Hoffman and Kyle Kolodziej

## 1. Problem Definition

For this project we would like to take the most recent world cup data (including each team that is playing in the tournament and the different statistical evaluations for these teams) and use this data to predict the percentage for success that a user imputed team has.  The user will be able to input a specific team and different attack and defense ratings for that team and the model will be able to remove that team from the dataset and predict the success percentage for that team given the inputed data.  This will allow a person to see the success percentages of their chosen team given its current data or in a hypothetical situation where its attack or defense is better than it actually is.

## 2. Data Gathering

## 3. Data Preperation and EDA

## 4. Feature Engineering and Feature Selection

## 5. Model Training and Model Evaluation

## 6. Hyperparameter Tuning and Evaluation

## 7. Prediction

## 8. References

[1] Deploying Machine Learning Model Using Heroku. https://datamahadev.com/deploying-machine-learning-model-using-heroku/

[2] Using Machine Learning to Simulate World Cup Matches. https://towardsdatascience.com/using-machine-learning-to-simulate-world-cup-matches-959e24d0731